In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from os import listdir
from os.path import isfile, join, isdir
import re
from matplotlib import pyplot as plt
import seaborn as sns
import glob
import re
from scipy.interpolate import interp1d
from tqdm import tqdm, tqdm_notebook
from scipy import spatial

In [2]:
def calculate_pad(brightness, saturation):
    p = 0.69*brightness + 0.22*saturation
    a = -0.31*brightness + 0.6*saturation
    d = 0.76*brightness + 0.32*saturation
    return [p,a,d]

def calculate_pad_scene(scene):
    pads = []
    for img in scene:
        temp_b = mean_brightness(img)
        temp_s = mean_saturation(img)
        pads.append(calculate_pad(temp_b, temp_s))
    return np.mean([x[0] for x in pads]), np.mean([x[1] for x in pads]), np.mean([x[2] for x in pads])

def calculate_blur(img):
    return cv2.Laplacian(img, cv2.CV_64F).var()

def calculate_blur_scene(scene):
    blurs = []
    for img in scene:
        blurs.append(calculate_blur(img))
    return np.mean(blurs)

def mean_brightness(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    return np.mean(hsv[:,:,2])

def mean_saturation(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    return np.mean(hsv[:,:,1])

def calculate_opticalFlow(img1, img2):
    prev = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    forward = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(prev)
    mask[..., 1] = 255
    flow = cv2.calcOpticalFlowFarneback(prev, forward, flow=None, pyr_scale=0.5,levels =1,winsize=3,iterations=15,poly_n=3,poly_sigma=5,flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    return cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)[0]

def calculate_opticalFlow_scene(scene):
    first = calculate_opticalFlow(scene[0], scene[1])
    second = calculate_opticalFlow(scene[1], scene[2])
    return np.mean([first, second])
    
def isjpg(filepath):
    return re.search(".jpg$", filepath)

In [5]:
df = pd.read_csv('clean_df.csv')
df.head()

,scene_name,scene_avg_p,scene_avg_a,scene_avg_d,scene_avg_blur,scene_avg_optical_flow,scene_movie
0,big_hero_60,64.405538,72.981351,82.268261,204.102944,4.284929e-08,big_hero_6
1,big_hero_61,76.658546,45.019868,93.268124,2058.214860,4.400276e-02,big_hero_6
2,big_hero_610,93.844222,48.897762,113.486587,319.563208,7.499705e-03,big_hero_6
3,big_hero_611,104.689522,68.900088,128.197153,344.318350,3.311605e-06,big_hero_6
4,big_hero_612,52.825514,48.692155,66.235224,302.668920,1.479887e-03,big_hero_6


In [10]:
df_pad = df[['scene_avg_p', 'scene_avg_a', 'scene_avg_d']]
df_pad_norm = (df_pad - df_pad.min()) / (df_pad.max() - df_pad.min())

In [14]:
ANGER_POINT = [-0.43, 0.67, 0.34]
JOY_POINT = [0.76, 0.48, 0.35]
SURPRISE_POINT = [0.4, 0.67, -0.13]
DISGUST_POINT = [-0.6, 0.35, 0.11]
FEAR_POINT = [-0.64, 0.6, -0.43]
SADNESS_POINT = [-0.63, 0.27, -0.33]

kdtree = spatial.cKDTree(np.array([ANGER_POINT,JOY_POINT,SURPRISE_POINT,DISGUST_POINT,FEAR_POINT,SADNESS_POINT]))
mu = df_pad_norm.sample(1)
dist, ix = kdtree.query([mu['scene_avg_p'].iloc[0],mu['scene_avg_a'].iloc[0],mu['scene_avg_d'].iloc[0]], k=2)
kdtree.data[ix]

array([[ 0.76,  0.48,  0.35],
       [ 0.4 ,  0.67, -0.13]])